In [1]:
from dotenv import load_dotenv
import os
import psycopg2
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

# Load environment variables from .env
load_dotenv()


False

In [2]:
def get_jobs(keyword, num_jobs, verbose):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver
    service = Service(executable_path='./chromedriver.exe')
    options = webdriver.ChromeOptions()
    
    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    
    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(options=options, service=service)
    driver.set_window_size(1120, 1000)

    url = f'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="{keyword}"&sc.locationSeoString=Riyadh+%28Saudi+Arabia%29&locId=3110290&locT=C'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  # If true, should be still looking for new jobs.

        time.sleep(4)  # Let the page load

        # Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME, "selected").click()
        except:
            pass

        try:
            driver.find_element(By.CLASS_NAME, "ModalStyle__xBtn___29PT9").click()  # Close the sign-up modal
        except:
            pass

        # Going through each job in this page
        job_cards = driver.find_elements(By.CLASS_NAME, "jobCard")  # Updated to match the current HTML structure
        
        for job_card in job_cards:  
            print(f"Progress: {len(jobs)}/{num_jobs}")
            if len(jobs) >= num_jobs:
                break

            try:
                job_title = job_card.find_element(By.CLASS_NAME, "JobCard_jobTitle___7I6y").text
                company_name = job_card.find_element(By.CLASS_NAME, "EmployerProfile_compactEmployerName__LE242").text
                location = job_card.find_element(By.CLASS_NAME, "JobCard_location__rCz3x").text
                job_description = job_card.find_element(By.CLASS_NAME, "JobCard_jobDescriptionSnippet__yWW8q").text
            except Exception as e:
                print(f"Failed to collect job data: {e}")
                continue

            try:
                salary_estimate = job_card.find_element(By.CLASS_NAME, "JobCard_salaryEstimate__arV5J").text
            except NoSuchElementException:
                salary_estimate = -1

            try:
                rating = job_card.find_element(By.CLASS_NAME, "EmployerProfile_ratingContainer__ul0Ef").text
            except NoSuchElementException:
                rating = -1

            # Printing for debugging
            if verbose:
                print(f"Job Title: {job_title}")
                print(f"Salary Estimate: {salary_estimate}")
                print(f"Job Description: {job_description[:500]}")
                print(f"Rating: {rating}")
                print(f"Company Name: {company_name}")
                print(f"Location: {location}")

            jobs.append({
                "Job Title": job_title,
                "Salary Estimate": salary_estimate,
                "Job Description": job_description,
                "Rating": rating,
                "Company Name": company_name,
                "Location": location
            })

        # Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, './/li[@class="next"]//a').click()
        except NoSuchElementException:
            print(f"Scraping terminated before reaching target number of jobs. Needed {num_jobs}, got {len(jobs)}.")
            break

    driver.quit()
    return pd.DataFrame(jobs)


In [3]:

def connect_db():
    print(os.getenv('DATABASE_URL'))
    return psycopg2.connect(os.getenv('DATABASE_URL'))


def convert_numpy_types(series):
    return series.map(lambda x: x.item() if isinstance(x, np.generic) else x)


In [4]:

# Function to insert job data into PostgreSQL
def insert_jobs_to_db(jobs_df):
    # Convert numpy types to native Python types for each column
    for col in jobs_df.columns:
        jobs_df[col] = convert_numpy_types(jobs_df[col])
    
    # Establish the connection
    conn = connect_db()
    cursor = conn.cursor()
    
    # Define the insert query
    insert_query = """
    INSERT INTO job_listings (job_title, salary_estimate, job_description, rating, company_name, location)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (job_title, company_name, location) DO NOTHING;
    """
    
    # Insert each row into the database
    for row in jobs_df.itertuples(index=False):
        print(f"Inserting row: {row[0]}")
        cursor.execute(insert_query, row)
    
    # Commit and close the connection
    conn.commit()
    cursor.close()
    conn.close()




In [5]:
import argparse
import sys

def parse_arguments():
    parser = argparse.ArgumentParser(description='Scrape Glassdoor for job listings.')
    parser.add_argument('job_title', type=str, help='Job title to search for')
    parser.add_argument('num_jobs', type=int, default=50, help='Number of jobs to scrape')
    parser.add_argument('--verbose', action='store_true', help='Increase output verbosity')
    return parser.parse_args()


if __name__ == "__main__":
    # args = parse_arguments()
    # df = get_jobs(args.job_title, args.num_jobs, args.verbose)
    df = get_jobs('data engineer', 20, True)

    insert_jobs_to_db(df)


WebDriverException: Message: 'chromedriver.exe' executable may have wrong permissions.
